In [1]:
import torch
from transformers import AutoTokenizer, AutoConfig, AutoModel, MT5Model

In [2]:
import os
from huggingface_hub import hf_hub_download

In [5]:
import os
from huggingface_hub import hf_hub_download
import torch
from transformers import AutoTokenizer, AutoConfig, AutoModel, MT5Model


class MT5EncoderForIntentRecognition(torch.nn.Module):
    def __init__(self, encoder, num_labels):
        super(MT5EncoderForIntentRecognition, self).__init__()
        self.encoder = encoder
        self.dropout = torch.nn.Dropout(0.1)
        self.classifier = torch.nn.Linear(self.encoder.config.d_model, num_labels)

    @classmethod
    def from_pretrained(cls, model_name, num_labels):
        config = AutoConfig.from_pretrained(model_name)
        encoder = MT5Model.from_pretrained(model_name).encoder
        model_path = hf_hub_download(repo_id=model_name, filename="pytorch_model.bin")

        model = cls(encoder, num_labels)
        state_dict = torch.load(model_path, map_location=torch.device('cuda' if torch.cuda.is_available() else 'cpu'))
        model.encoder.load_state_dict(state_dict['encoder_state_dict'], strict=False)
        model.classifier.load_state_dict(state_dict['classifier_state_dict'], strict=False)

        return model

    def forward(self, input_ids, attention_mask=None, labels=None):
        encoder_outputs = self.encoder(input_ids=input_ids, attention_mask=attention_mask)
        sequence_output = encoder_outputs.last_hidden_state
        pooled_output = sequence_output[:, 0, :]
        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)

        loss = None
        if labels is not None:
            loss_fct = torch.nn.CrossEntropyLoss()
            loss = loss_fct(logits.view(-1, self.classifier.out_features), labels.view(-1))

        return {"loss": loss, "logits": logits} if loss is not None else {"logits": logits}


In [7]:
model_name = "NguyenMinh03082004/intent_recognition_based_mt5_encoder"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = MT5EncoderForIntentRecognition.from_pretrained(model_name, num_labels=150)
device = torch.device('cpu')
model.to(device)

c:\Users\ADMIN\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\convert_slow_tokenizer.py:561: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


config.json:   0%|          | 0.00/774 [00:00<?, ?B/s]

c:\Users\ADMIN\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\ADMIN\.cache\huggingface\hub\models--NguyenMinh03082004--intent_recognition_based_mt5_encoder. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model.safetensors:   0%|          | 0.00/588M [00:00<?, ?B/s]

Some weights of MT5Model were not initialized from the model checkpoint at NguyenMinh03082004/intent_recognition_based_mt5_encoder and are newly initialized: ['decoder.block.0.layer.0.SelfAttention.k.weight', 'decoder.block.0.layer.0.SelfAttention.o.weight', 'decoder.block.0.layer.0.SelfAttention.q.weight', 'decoder.block.0.layer.0.SelfAttention.relative_attention_bias.weight', 'decoder.block.0.layer.0.SelfAttention.v.weight', 'decoder.block.0.layer.0.layer_norm.weight', 'decoder.block.0.layer.1.EncDecAttention.k.weight', 'decoder.block.0.layer.1.EncDecAttention.o.weight', 'decoder.block.0.layer.1.EncDecAttention.q.weight', 'decoder.block.0.layer.1.EncDecAttention.v.weight', 'decoder.block.0.layer.1.layer_norm.weight', 'decoder.block.0.layer.2.DenseReluDense.wi_0.weight', 'decoder.block.0.layer.2.DenseReluDense.wi_1.weight', 'decoder.block.0.layer.2.DenseReluDense.wo.weight', 'decoder.block.0.layer.2.layer_norm.weight', 'decoder.block.1.layer.0.SelfAttention.k.weight', 'decoder.block.1

pytorch_model.bin:   0%|          | 0.00/588M [00:00<?, ?B/s]

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_15904\132887187.py:21: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(model_path, map_location=torch.devic

MT5EncoderForIntentRecognition(
  (encoder): MT5Stack(
    (embed_tokens): Embedding(250112, 512)
    (block): ModuleList(
      (0): MT5Block(
        (layer): ModuleList(
          (0): MT5LayerSelfAttention(
            (SelfAttention): MT5Attention(
              (q): Linear(in_features=512, out_features=384, bias=False)
              (k): Linear(in_features=512, out_features=384, bias=False)
              (v): Linear(in_features=512, out_features=384, bias=False)
              (o): Linear(in_features=384, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 6)
            )
            (layer_norm): MT5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): MT5LayerFF(
            (DenseReluDense): MT5DenseGatedActDense(
              (wi_0): Linear(in_features=512, out_features=1024, bias=False)
              (wi_1): Linear(in_features=512, out_features=1024, bias=False)
              (wo): Linear(in_features=102

In [8]:
import pandas as pd
data_test = pd.read_json('is_test.json')

In [9]:
print(data_test)

                                                 0              1
0                 how would you say fly in italian      translate
1                what's the spanish word for pasta      translate
2              how would they say butter in zambia      translate
3                   how do you say fast in spanish      translate
4              what's the word for trees in norway      translate
...                                            ...            ...
4495                why can't i use my credit card  card_declined
4496  why won't you let me pay with my credit card  card_declined
4497             why did i get rejected on my card  card_declined
4498         how come my credit card isn't working  card_declined
4499                       why didn't my card work  card_declined

[4500 rows x 2 columns]


In [10]:
x_test = data_test[0].values
y_test = data_test[1].values

In [12]:
print(x_test)
print(y_test)

['how would you say fly in italian' "what's the spanish word for pasta"
 'how would they say butter in zambia' ...
 'why did i get rejected on my card'
 "how come my credit card isn't working" "why didn't my card work"]
['translate' 'translate' 'translate' ... 'card_declined' 'card_declined'
 'card_declined']


In [13]:
import numpy as np
unique_labels = np.unique(y_test)

In [14]:
print(unique_labels)

['accept_reservations' 'account_blocked' 'alarm' 'application_status'
 'apr' 'are_you_a_bot' 'balance' 'bill_balance' 'bill_due' 'book_flight'
 'book_hotel' 'calculator' 'calendar' 'calendar_update' 'calories'
 'cancel' 'cancel_reservation' 'car_rental' 'card_declined' 'carry_on'
 'change_accent' 'change_ai_name' 'change_language' 'change_speed'
 'change_user_name' 'change_volume' 'confirm_reservation' 'cook_time'
 'credit_limit' 'credit_limit_change' 'credit_score' 'current_location'
 'damaged_card' 'date' 'definition' 'direct_deposit' 'directions'
 'distance' 'do_you_have_pets' 'exchange_rate' 'expiration_date'
 'find_phone' 'flight_status' 'flip_coin' 'food_last' 'freeze_account'
 'fun_fact' 'gas' 'gas_type' 'goodbye' 'greeting' 'how_busy'
 'how_old_are_you' 'improve_credit_score' 'income'
 'ingredient_substitution' 'ingredients_list' 'insurance'
 'insurance_change' 'interest_rate' 'international_fees'
 'international_visa' 'jump_start' 'last_maintenance' 'lost_luggage'
 'make_call'

In [19]:
label_to_num = {}
for i in range(len(unique_labels)):
    label_to_num[unique_labels[i]] = i 
num_to_label = {}
for i in range(len(unique_labels)):
    num_to_label[i] = unique_labels[i]


In [20]:
print(label_to_num)
print(num_to_label)

{'accept_reservations': 0, 'account_blocked': 1, 'alarm': 2, 'application_status': 3, 'apr': 4, 'are_you_a_bot': 5, 'balance': 6, 'bill_balance': 7, 'bill_due': 8, 'book_flight': 9, 'book_hotel': 10, 'calculator': 11, 'calendar': 12, 'calendar_update': 13, 'calories': 14, 'cancel': 15, 'cancel_reservation': 16, 'car_rental': 17, 'card_declined': 18, 'carry_on': 19, 'change_accent': 20, 'change_ai_name': 21, 'change_language': 22, 'change_speed': 23, 'change_user_name': 24, 'change_volume': 25, 'confirm_reservation': 26, 'cook_time': 27, 'credit_limit': 28, 'credit_limit_change': 29, 'credit_score': 30, 'current_location': 31, 'damaged_card': 32, 'date': 33, 'definition': 34, 'direct_deposit': 35, 'directions': 36, 'distance': 37, 'do_you_have_pets': 38, 'exchange_rate': 39, 'expiration_date': 40, 'find_phone': 41, 'flight_status': 42, 'flip_coin': 43, 'food_last': 44, 'freeze_account': 45, 'fun_fact': 46, 'gas': 47, 'gas_type': 48, 'goodbye': 49, 'greeting': 50, 'how_busy': 51, 'how_ol

In [22]:
y_test = np.vectorize(label_to_num.get)(y_test)

In [23]:
data_test = {'message': x_test, 'intent': y_test}

In [24]:
print(data_test)

{'message': array(['how would you say fly in italian',
       "what's the spanish word for pasta",
       'how would they say butter in zambia', ...,
       'why did i get rejected on my card',
       "how come my credit card isn't working", "why didn't my card work"],
      dtype=object), 'intent': array([131, 131, 131, ...,  18,  18,  18])}


In [25]:
dataset_test = pd.DataFrame(data_test)

In [26]:
def tokenize_function(examples):
    return tokenizer(examples["message"], padding="max_length", truncation=True, max_length=28)
test_encodings = tokenizer(list(dataset_test['message']), truncation=True, padding=True)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [27]:
# convert datasets to pytorch tensors
class IntentDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item
    def __len__(self):
        return len(self.labels)

In [28]:
test_dataset = IntentDataset(test_encodings, list(dataset_test['intent']))

In [29]:
from transformers import DataCollatorWithPadding


In [30]:
def predict_intent(text, model, tokenizer, num_to_label):
    inputs = tokenizer(text, return_tensors="pt", padding="max_length", truncation=True, max_length=128)
    inputs = {key: val.to(device) for key, val in inputs.items()}

    model.eval()
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs['logits']
        predictions = torch.argmax(logits, dim=-1)

    return num_to_label[predictions.item()]

# Example usage
input_text = "Do you know hello in Vietnamese is"
predicted_intent = predict_intent(input_text, model, tokenizer, num_to_label)
print(f"Predicted intent: {predicted_intent}")


Predicted intent: translate
